In [1]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# device = 'cuda:0'
device = 0
# G = Generator(32,256,512,32).eval().to(device)
G = Generator(16,256,512,16).eval().to(device) # change the dimension to the same as model config from parser

autovc_model = '/home/ytang363/7100_voiceConversion/trained_model.ckpt'
g_checkpoint = torch.load(autovc_model, map_location=torch.device(device))
print(g_checkpoint.keys())
G.load_state_dict(g_checkpoint['model'])

# metadata contains [filename, speaker embedding, spectrogram]
metadata = pickle.load(open('metadata.pkl', "rb"))
spect_vc = []   

dict_keys(['model', 'optimizer'])


FileNotFoundError: [Errno 2] No such file or directory: 'metadata.pkl'

In [2]:
for sbmt_i in metadata:
    print(sbmt_i[0])

    x_org = sbmt_i[2] # spectrogram
    x_org, len_pad = pad_seq(x_org)
    print(x_org.shape)

    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device) # spectrogram
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device) # speaker embedding
    print(uttr_org.shape)
    
    for sbmt_j in metadata:
        # print(sbmt_j)

        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else: # excluding a padding portion
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        # print(spect_vc)
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)       

p225
(96, 80)
torch.Size([1, 96, 80])
p228
(96, 80)
torch.Size([1, 96, 80])
p256
(96, 80)
torch.Size([1, 96, 80])
p270
(128, 80)
torch.Size([1, 128, 80])


In [3]:
res = pickle.load(open('results.pkl', "rb"))
for i in res:
    print(i[0])
    print(i[1].shape)

p225xp225
(90, 80)
p225xp228
(90, 80)
p225xp256
(90, 80)
p225xp270
(90, 80)
p228xp225
(89, 80)
p228xp228
(89, 80)
p228xp256
(89, 80)
p228xp270
(89, 80)
p256xp225
(75, 80)
p256xp228
(75, 80)
p256xp256
(75, 80)
p256xp270
(75, 80)
p270xp225
(109, 80)
p270xp228
(109, 80)
p270xp256
(109, 80)
p270xp270
(109, 80)
